# TensorRTへの変換

PyTorchから、TensorRTへの変換

train_modelで生成した学習済みモデルである'best_steering_model_xy.pth'を読み込みます。

In [ ]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_xy.pth'))

以下のコードを実行してGPUデバイスに転送します。

In [ ]:
device = torch.device('cuda')
model = model.to(device)
model = model.cuda().eval().half()

TorchからTensorRTに変換します。

In [ ]:
from torch2trt import torch2trt 
data = torch.randn((1, 3, 224, 224)).cuda().half()
model_trt = torch2trt(model, [data], fp16_mode=True)

変換が正しく行われたか、変換前と変換後の比較をおこないます。

In [ ]:
output_trt = model_trt(data)
output = model(data)

print(output.flatten()[0:10])
print(output_trt.flatten()[0:10])
print('max error: %f' % float(torch.max(torch.abs(output - output_trt))))

TensorRTのモデルとして保存。

In [ ]:
torch.save(model_trt.state_dict(), 'road_following_model_xy_trt.pth')